In [54]:
# Scrapping from reclameaqui.com.br
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
import os

from selenium.webdriver.common.by import By


In [55]:
# nome da empresa seguindo convenção do site reclameaqui.com.br
empresa : str = 'temu'

# URL da pagina principal da empresa
url_p = 'https://www.reclameaqui.com.br/empresa/' + empresa + '/'

# URL da pagina de reclamações da empresa
url_r = 'https://www.reclameaqui.com.br/empresa/' + empresa + '/lista-reclamacoes/?status=ANSWERED'



In [56]:
# Configurações do navegador
ff_options = Options()
ff_options.headless = False

# Configurações do serviço
if os.name == 'nt': # Windows
    service = Service('./geckodriver.exe')
else: # Linux
    service = Service('./geckodriver')





In [57]:
# pegando metadados da empresa (nome, categoria, reputação, sobre)
try:

   # driver
   driver = webdriver.Firefox(options=ff_options, service=service)
   
   # acessando a pagina principal da empresa
   driver.get(url_p)
      
   # ramo de atividade da empresa
   categoria = driver.find_element(By.ID, 'info_segmento_hero').text

   # reputação da empresa
   reputacao = driver.find_element(By.ID, 'tag_reputacao_hero').text

   # sobre a empresa
   sobre = driver.find_element(By.CSS_SELECTOR, 'ul.sc-1915fv4-2.idgfcC').text
   sobre = sobre.split("\n")[0] 

   metadata = {
      'nome': empresa,
      'categoria': categoria,         
      'reputacao': reputacao,         
      'sobre': sobre       
   }

finally:

   driver.quit()


print(metadata)

{'nome': 'temu', 'categoria': 'Varejo', 'reputacao': 'Não Recomendada', 'sobre': 'A Temu é uma empresa de comércio eletrônico que conecta consumidores a milhões de parceiros de mercadorias, fabricantes e marcas com a missão de capacitá-los a viver uma vida melhor.'}


In [ ]:
# pegando hyperlinks das reclamações respondidas
try:
    # driver
    driver = webdriver.Firefox(options=ff_options, service=service)
    
    # acessando a pagina de reclamações respondidas
    driver.get(url_r)
        
    # numero de abas de reclamações respondidas
    num_abas = driver.find_elements(By.CSS_SELECTOR, 'span.sc-fAGzit.XgFJr')

    print(num_abas)

finally:
    print('fim')
    #driver.quit()

[<selenium.webdriver.remote.webelement.WebElement (session="114187ea-7d57-4642-a3a9-4ee5647908c7", element="742704a1-5110-4a75-a78b-02c613758aa2")>]
fim


In [ ]:
# pegando dados das reclamações respondidas
try:
    # driver
    driver = webdriver.Firefox(options=ff_options, service=service)
    
    

finally:
   
    driver.quit()

AttributeError: 'list' object has no attribute 'text'